In [1]:
import sys
sys.path.insert(0, "..")

In [2]:
import spacy
import medspacy

# Overview
In this notebook, we'll look at two of the first steps commonly performed on clinical text and see how medSpaCy handles them:
- Tokenization
- Sentence splitting

In [3]:
with open("./discharge_summary.txt") as f:
    text = f.read()

In [4]:
# Use a blank model rather than the full medspaCy pipeline
nlp = spacy.blank("en")

# Tokenization
Clinical language is very different from general natural language. Abbreviations and punctuation in particularly are used irregularly and tokenizers trained on general English sources like Wikipedia perform poorly on clinical text.

To address this, medSpaCy adds a more aggressive tokenizer with rules specifically meant to handle clinical text. This is loaded by default with the `medspacy.load()` function, but can also be created using this utility function:

In [5]:
from medspacy.custom_tokenizer import create_medspacy_tokenizer

In [6]:
medspacy_tokenizer = create_medspacy_tokenizer(nlp)
default_tokenizer = nlp.tokenizer

In [7]:
example_text = r'Pt c\o n;v;d h\o chf+cp'

In [8]:
print("Tokens from default tokenizer:")
print(list(default_tokenizer(example_text)))
print()
print("Tokens from medspacy tokenizer:")
print(list(medspacy_tokenizer(example_text)))

Tokens from default tokenizer:
[Pt, c\o, n;v;d, h\o, chf+cp]

Tokens from medspacy tokenizer:
[Pt, c, \, o, n, ;, v, ;, d, h, \, o, chf, +, cp]


Now we'll add our custom tokenizer to our pipeline by replacing the default:

In [9]:
nlp.tokenizer = medspacy_tokenizer

In [10]:
print(list(nlp(example_text)))

[Pt, c, \, o, n, ;, v, ;, d, h, \, o, chf, +, cp]


# Sentence segmentation
Sentence segmentation in medSpaCy is performed in one of several ways: 
* Through the standard part-of-speech tagger `tagger` and dependency parser `parser` implemented by default in a spaCy [model](https://spacy.io/models), such as the `en_core_web_sm` model. This is not designed for clinical text and the dependency parsing algorithm can be both unnecessary and slow down your pipeline.
* Through the `sentencizer` component as part of a spaCy model that is not enabled by default. This speeds up the pipeline by allowing the removal of the `tagger` and `parser` components, but is still not trained on clinical text.
* Using [PyRuSH](https://github.com/jianlins/PyRuSH). This is a rule based sentence splitter designed for clinical text. 


PyRuSH rules are defined by a resources file. PyRuSH is included by default in `medspacy.load()` but, like all other medspaCy components, can be added to a spaCy pipeline individually.

## PyRuSH

In [11]:
from medspacy.sentence_splitting import PyRuSHSentencizer

In [12]:
# For new version of sentencizer in PyRuSH, it is embedded in pipline
#sentencizer = PyRuSHSentencizer(rules_path="../resources/rush_rules.tsv")
#sentencizer

In [13]:
nlp.add_pipe("medspacy_pyrush") #, config={"pyrush_path": "../resources/rush_rules.tsv"})

In [14]:
nlp.pipe_names

['medspacy_pyrush']

In [15]:
doc = nlp(text)

In [16]:
for sent in doc.sents:
    print(sent)
    print("---"*10)

Admission Date:  [**2573-5-30**]              
------------------------------
Discharge Date:   [**2573-7-1**]


------------------------------
Date of Birth:  [**2498-8-19**]             
------------------------------
Sex:   F


------------------------------
Service: SURGERY


------------------------------
Allergies:

------------------------------
Hydrochlorothiazide


------------------------------
Attending:[**First Name3 (LF) 1893**]

------------------------------
Chief Complaint:

------------------------------
Abdominal pain


------------------------------
Major Surgical or Invasive Procedure:

------------------------------
PICC line [**6-25**]

------------------------------
ERCP w/ sphincterotomy [**5-31**]



------------------------------
History of Present Illness:

------------------------------
74y female with type 2 dm and a recent stroke affecting her
speech, who presents with 2 days of abdominal pain.
------------------------------
Imaging shows no evidence of me